In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import librosa
import numpy as np

In [ ]:
def read_csv(path):
    return pd.read_csv(path)

In [ ]:
def split_times(arr,ratio):
    arr = np.array(arr)
    proportions = arr/arr.sum()
    if not type(ratio)==float:
        #for id in 
        proportions[np.where(np.array(ratio)==1)[0]]=1
    test_times = arr*proportions*ratio
    return test_times

In [ ]:
def student_wise_distribution(df):
    count=[]
    dur=[]
    filenames=[]
    a = df['s_id'].unique()
    for i in a:
        count.append(df['s_id'].value_counts()[i])
        l=df[df['s_id']==i]['duration']
        dur.append(sum(l.to_list()))
        f=df[df['s_id']==i]['s_file']
        filenames.append(f.to_list())
    return a,count,dur,filenames

In [ ]:
def get_filenames_student_wise(df,s_id):
        l=df[df['s_id']==s_id]['s_file']
        return sort(l.to_list())

In [ ]:
def get_all_filenames(df,ids):
    lst=[]
    for i in ids:
        l=df[df['s_id']==i]['s_file']
        lst.append(l.to_list())
    return lst

In [ ]:
def sort_data(ids,dur,count,times,f):
    d,c,i,t,f = zip(*sorted(zip(dur,count,ids,times,f)))
    d,c,i,t,f=d[::-1],c[::-1],i[::-1],t[::-1],f[::-1]
    return d,c,t,i,f

In [ ]:
def get_test_files_student_wise(f,df,t):
    lst=[]
    for i in range(len(f)):
        s=0
        l=[]
        for j in range(len(f[i])):
            d=float(df[df['s_file']==f[i][-(j+1)]]['duration'])
            s+=d
            l.append(f[i][-(j+1)])
            if s>=t[i]:
                break
        lst.append(l)   
    return lst

In [ ]:
def ids_for_test(test_ratio,ids,dur,total_dur):
    rem_dur=test_ratio*total_dur
    dur,ids = zip(*sorted(zip(dur,ids)))
    lst=[]
    s=0
    for i in range(len(dur)):
        s+=dur[i]
        lst.append(ids[i])
        if s>=rem_dur:
            break
    return lst 

In [ ]:
def update_dataframe(df,lst):
    for i in lst:
        df = df[df.s_id != i]
    return df

In [ ]:
def save_train_data(df,t_id):
    df = df.iloc[: , 2:]
    df.to_csv("train_data_"+str(t_id)+".csv",index=False)
    print("Train Data Saved")

In [ ]:
def merge_list(arr):
    new_arr=[]
    for i in arr:
        for j in i:
            new_arr.append(j)
    return new_arr

In [ ]:
def save_test_data(df,arr,t_id,t):
    lst=merge_list(arr)
    ndf = pd.DataFrame()
    for i in lst:
        x=df.loc[df['s_file'] == i]
        ndf = ndf.append(x, ignore_index = True)
    ndf = df.iloc[: , 2:]
    if t=="":
        ndf.to_csv("test_data_"+str(t_id)+".csv",index=False)
    else:
         ndf.to_csv("test_data_"+str(t)+"_"+str(t_id)+".csv",index=False)
    print("Test Data Saved")    

In [ ]:
metadata_path_t1="metadata_001_01-03-2023.csv"
metadata_path_t2="metadata_002_01-03-2023.csv"

In [ ]:
df1=read_csv(metadata_path_t1)
df2=read_csv(metadata_path_t2)


a,a_count,a_dur,f1=student_wise_distribution(df1)
b,b_count,b_dur,f2=student_wise_distribution(df2)
total_dur1=sum(a_dur)
total_dur2=sum(b_dur)

test_ratio=0.15
ids1=ids_for_test(test_ratio,a,a_dur,total_dur1)
ids2=ids_for_test(test_ratio,b,b_dur,total_dur2)

test1=get_all_filenames(df1,ids1)
test2=get_all_filenames(df2,ids2)

# print(test1)
# print(test2)

save_test_data(df1,test1,"001","")
save_test_data(df2,test2,"002","")




ndf1=update_dataframe(df1,ids1)
ndf2=update_dataframe(df2,ids2)


save_train_data(ndf1,"001")
save_train_data(ndf2,"002")

a,a_count,a_dur,f1=student_wise_distribution(ndf1)
b,b_count,b_dur,f2=student_wise_distribution(ndf2)
total_dur1=sum(a_dur)
total_dur2=sum(b_dur)

a_times=split_times(a_dur,ratio=0.2)
b_times=split_times(b_dur,ratio=0.2)

d1,c1,t1,i1,f1=sort_data(a,a_dur,a_count,a_times,f1)
d2,c2,t2,i2,f2=sort_data(b,b_dur,b_count,b_times,f2)

tf1=get_test_files_student_wise(f1,df1,t1)
tf2=get_test_files_student_wise(f2,df2,t2)

save_test_data(ndf1,tf1,"001","1")
save_test_data(ndf2,tf2,"002","1")

# print(tf1)
# print(tf2)
